# Random Forest Classification via cross-validation

- classification task: SL (1) or not SL (0)
- classification tree can handle all types of data, all types of relationships among the independent variables (the data we're using to make predictions), and all kinds of relationships with the dependent variable (the thing we want to predict)
- The column named SL is my target variable (i.e., the variable which I want to predict). There are two possible classes: 0 (non-SL) and 1 (SL). The resulting prediction problem is therefore a binary classification problem, while I will use the other columns (feature columns) as input variables for the model.

*StratifiedKFold* is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set.

*StratifiedKFold* preserves the class ratios (approximately 1 / 10) in both train and test dataset.

*StratifiedGroupKFold* is used to generate the train and test sets for model 2 and model 3

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedGroupKFold.html#sklearn.model_selection.StratifiedGroupKFold

The implementation is designed to:
- Generate test sets such that all contain the same distribution of classes, or as close as possible.
- Be invariant to class label: relabelling y = ["Happy", "Sad"] to y = [1, 0] should not change the indices generated.
- Preserve order dependencies in the dataset ordering, when shuffle=False: all samples from class k in some test set were contiguous in y, or separated in y by samples from classes other than k.
- Generate test sets where the smallest and largest differ by at most one sample.

*type_of_target* : Determine the type of data indicated by the target, binary, multiclass, etc. 

In [80]:
# import modules
import os
import pandas as pd
import numpy as np
import random
import pickle

# import scikit-learn modules
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, roc_auc_score, average_precision_score, precision_recall_curve, auc, roc_curve

# import visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [81]:
# save path
get_data_path = lambda folders, fname: os.path.normpath(os.environ['DRIVE_PATH'] + '/' + '/'.join(folders) + '/' + fname)

file_path = get_data_path(['SL PRED', 'NEW'], 'ito_pairs_annotated.csv')
old_classifier_path = get_data_path(['SL PRED', 'input'], 'processed_DeKegel_TableS8.csv')

# Output
file_RF_model_I_early = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_I'], 'RF_model_early.pickle')
file_RF_model_I_late = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_I'], 'RF_model_late.pickle')
#file_RF_model_I_early_repeated = get_data_path(['SL PRED', 'output', 'NEW', 'cross_validation', 'model_I'], 'RF_model_early_repeated.pickle')
#file_RF_model_I_late_repeated = get_data_path(['SL PRED', 'output', 'NEW', 'cross_validation', 'model_I'], 'RF_model_late_repeated.pickle')
#file_RF_model_I_early_reduced = get_data_path(['SL PRED', 'output', 'NEW', 'cross_validation', 'model_I'], 'RF_model_early_reduced.pickle')
#file_RF_model_I_late_reduced = get_data_path(['SL PRED', 'output', 'NEW', 'cross_validation', 'model_I'], 'RF_model_late_reduced.pickle')

file_RF_model_II_early = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_II'], 'RF_model_early.pickle')
file_RF_model_II_late = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_II'], 'RF_model_late.pickle')
#file_RF_model_II_early_repeated = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_II'], 'RF_model_early_repeated.pickle')
#file_RF_model_II_late_repeated = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_II'], 'RF_model_late_repeated.pickle')
#file_RF_model_II_early_reduced = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_II'], 'RF_model_early_reduced.pickle')
#file_RF_model_II_late_reduced= get_data_path(['SL PRED', 'output', 'cross_validation', 'model_II'], 'RF_model_late_reduced.pickle')

file_RF_model_III_early = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_III'], 'RF_model_early.pickle')
file_RF_model_III_late = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_III'], 'RF_model_late.pickle')
#file_RF_model_III_early_repeated = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_III'], 'RF_model_early_repeated.pickle')
#file_RF_model_III_late_repeated = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_III'], 'RF_model_late_repeated.pickle')
#file_RF_model_III_early_reduced = get_data_path(['SL PRED', 'output', 'cross_validation', 'model_III'], 'RF_model_early_reduced.pickle')
#file_RF_model_III_late_reduced= get_data_path(['SL PRED', 'output', 'cross_validation', 'model_III'], 'RF_model_late_reduced.pickle')

file_RF_model_IV_early = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_IV'], 'RF_model_early.pickle')
file_RF_model_IV_late = get_data_path(['SL PRED', 'NEW', 'imputation', 'cross_validation', 'model_IV'], 'RF_model_late.pickle')


In [82]:
training_df = pd.read_csv(file_path)
training_df.head()

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,A2_rank,zA2_rank,max_ranked_A1A2,min_ranked_A1A2,z_max_ranked_A1A2,z_min_ranked_A1A2,prediction_score,GEMINI,LFC,SL_new
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,NaN,NaN,5108.0,5108.0,-0.759628,-0.759628,0.012559,0.118768,0.088856,False
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,NaN,NaN,9125.0,9125.0,0.303431,0.303431,0.012559,0.132501,0.201704,False
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,NaN,NaN,4063.0,4063.0,-1.036177,-1.036177,0.012559,0.024593,0.069772,False
3,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000681,A549_LUNG,0.977988,0.379455,False,...,NaN,NaN,15434.0,15434.0,1.973047,1.973047,0.012559,-0.241323,0.379455,False
4,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000756,GI1_CENTRAL_NERVOUS_SYSTEM,0.999586,-0.077118,False,...,NaN,NaN,6507.0,6507.0,-0.389397,-0.389397,0.012559,0.299715,-0.077118,False


In [83]:
old_classifier_training_df = pd.read_csv(old_classifier_path)
old_classifier_training_df[:3]

,prediction_rank,prediction_percentile,old_genepair,genepair,A1,A2,A1_entrez,A2_entrez,A1_ensembl,A2_ensembl,...,shared_ppi_mean_essentiality,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,A1_entrez_new,A2_entrez_new,A1_new,A2_new,A1_ensembl_new,A2_ensembl_new
0,1,0.1,SMARCA2_SMARCA4,SMARCA2_SMARCA4,SMARCA2,SMARCA4,6595,6597,ENSG00000080503,ENSG00000127616,...,0.225382,0.627875,18.609973,34.302868,6595.0,6597,SMARCA2,SMARCA4,ENSG00000080503,ENSG00000127616
1,2,0.1,EXOC6_EXOC6B,EXOC6_EXOC6B,EXOC6,EXOC6B,54536,23233,ENSG00000138190,ENSG00000144036,...,0.285886,0.069456,6.390812,11.168367,54536.0,23233,EXOC6,EXOC6B,ENSG00000138190,ENSG00000144036
2,3,0.1,STAG1_STAG2,STAG1_STAG2,STAG1,STAG2,10274,10735,ENSG00000118007,ENSG00000101972,...,0.329993,0.854086,13.103716,22.097616,10274.0,10735,STAG1,STAG2,ENSG00000118007,ENSG00000101972


In [84]:
late_features_df = old_classifier_training_df[['genepair', 'A1_entrez_new', 'A2_entrez_new', 'min_sequence_identity', 'closest', 'WGD', 'family_size', 'cds_length_ratio', 'shared_domains', 'has_pombe_ortholog',
                                            'has_essential_pombe_ortholog', 'has_cerevisiae_ortholog', 'has_essential_cerevisiae_ortholog', 'conservation_score', 'mean_age', 'either_in_complex', 'mean_complex_essentiality', 'colocalisation',
                                            'interact', 'n_total_ppi', 'fet_ppi_overlap','gtex_spearman_corr', 'gtex_min_mean_expr', 'gtex_max_mean_expr']]
late_features_df = late_features_df.rename(columns={'A1_entrez_new':'A1_entrez', 'A2_entrez_new': 'A2_entrez'})

In [85]:
def integrate_features(df, features_df):
    integrated_df = pd.merge(df, features_df, 
                             left_on=['genepair', 'A1_entrez', 'A2_entrez'], 
                             right_on=['genepair', 'A1_entrez', 'A2_entrez'], 
                             how='left')
    bool_cols = ['closest', 'WGD', 'has_pombe_ortholog', 'has_essential_cerevisiae_ortholog', 'either_in_complex', 'interact']
    integrated_df[bool_cols] = integrated_df[bool_cols].astype(bool)
    return integrated_df

In [86]:
integrated_training_df = integrate_features(training_df, late_features_df)
integrated_training_df[:3]

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,mean_age,either_in_complex,mean_complex_essentiality,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702


In [87]:
feature_columns_1 = ['rMaxExp_A1A2', 'rMinExp_A1A2',
                     'max_ranked_A1A2', 'min_ranked_A1A2',
                     'max_cn', 'min_cn', 'Protein_Altering', 'Damaging', 
                     'min_sequence_identity',
                     'prediction_score', 
                     'ranked_Essentiality_weighted_PPI', 'Expression_weighted_PPI',
                     'smallest_GO_ranked_ess', 'smallest_GO_CC_ranked_ess',
                     'smallest_gene_expression', 'smallest_GO_CC_expression'
                     ]

target_column = 'SL_new'

print('num of features:', len(feature_columns_1))

num of features: 16


In [88]:
feature_columns_2 = feature_columns_1 + ['closest', 'WGD', 'family_size',
                                         'cds_length_ratio', 'shared_domains', 'has_pombe_ortholog',
                                         'has_essential_pombe_ortholog', 'has_cerevisiae_ortholog', 'has_essential_cerevisiae_ortholog', 
                                         'conservation_score', 'mean_age', 'either_in_complex', 'mean_complex_essentiality', 'colocalisation',
                                         'interact', 'n_total_ppi', 'fet_ppi_overlap',
                                         'gtex_spearman_corr', 'gtex_min_mean_expr', 'gtex_max_mean_expr']
feature_columns_2.remove('prediction_score')
print('num of features:', len(feature_columns_2))

num of features: 35


In [10]:
#missing_values = integrated_training_df[feature_columns_1].isna().sum() / len(integrated_training_df) * 100
#missing_values

In [89]:
integrated_training_df[feature_columns_1].isna().sum()

rMaxExp_A1A2                           11
rMinExp_A1A2                           11
max_ranked_A1A2                      4926
min_ranked_A1A2                      4926
max_cn                                  5
min_cn                                  5
Protein_Altering                        0
Damaging                                0
min_sequence_identity                3347
prediction_score                     3347
ranked_Essentiality_weighted_PPI     7901
Expression_weighted_PPI              3793
smallest_GO_ranked_ess              26137
smallest_GO_CC_ranked_ess           40788
smallest_gene_expression            23970
smallest_GO_CC_expression           39944
dtype: int64

In [90]:
integrated_training_df[feature_columns_2].isna().sum()

rMaxExp_A1A2                            11
rMinExp_A1A2                            11
max_ranked_A1A2                       4926
min_ranked_A1A2                       4926
max_cn                                   5
min_cn                                   5
Protein_Altering                         0
Damaging                                 0
min_sequence_identity                 3347
ranked_Essentiality_weighted_PPI      7901
Expression_weighted_PPI               3793
smallest_GO_ranked_ess               26137
smallest_GO_CC_ranked_ess            40788
smallest_gene_expression             23970
smallest_GO_CC_expression            39944
closest                                  0
WGD                                      0
family_size                           3347
cds_length_ratio                      3347
shared_domains                        3347
has_pombe_ortholog                       0
has_essential_pombe_ortholog          3347
has_cerevisiae_ortholog               3347
has_essenti

In [91]:
def preprocess_dataset(df, old_df,
                       required_genepairs_col='genepair',
                       dropna_cols=None,
                       fillna_zero_cols=None,
                       fillna_large_cols=None,
                       fillna_large_value=18000):
    """
    Preprocess a training or testing dataset:
    - Keep only rows with genepairs present in `old_df`
    - Drop rows with NaN in specific columns
    - Fill missing values with default values
    
    Parameters:
        df (pd.DataFrame): Dataset to process
        old_df (pd.DataFrame): Dataset with allowed genepairs
        required_genepairs_col (str): Column name for genepairs to match
        dropna_cols (list): Columns for which rows with NaNs should be dropped
        fillna_zero_cols (list): Columns to fill NaNs with 0
        fillna_large_cols (list): Columns to fill NaNs with large constant
        fillna_large_value (int or float): The large value to fill (default: 18000)

    Returns:
        pd.DataFrame: Cleaned and processed DataFrame
    """

    # Step 1: Filter rows to only those in old_df
    df_filtered = df[df[required_genepairs_col].isin(old_df[required_genepairs_col])].copy()

    # Step 2: Drop rows with any NA in required columns
    if dropna_cols:
        df_filtered = df_filtered.dropna(axis=0, how='any', subset=dropna_cols)

    # Step 3: Fill NaNs with 0 or large number
    if fillna_zero_cols:
        df_filtered[fillna_zero_cols] = df_filtered[fillna_zero_cols].fillna(0)

    if fillna_large_cols:
        df_filtered[fillna_large_cols] = df_filtered[fillna_large_cols].fillna(fillna_large_value)

    # Step 4: Reset index for clean result
    return df_filtered.reset_index(drop=True)

In [95]:
drop_na_values = ['rMaxExp_A1A2', 'rMinExp_A1A2', 'max_ranked_A1A2', 'min_ranked_A1A2']
fillna_values = ['Expression_weighted_PPI', 'smallest_gene_expression', 'smallest_GO_CC_expression']
fillna_values_v2 = ['ranked_Essentiality_weighted_PPI', 'smallest_GO_ranked_ess', 'smallest_GO_CC_ranked_ess']

# Apply to training set
integrated_training_df_clean = preprocess_dataset(
    df=integrated_training_df,
    old_df=old_classifier_training_df,
    dropna_cols=drop_na_values,
    fillna_zero_cols=fillna_values,
    fillna_large_cols=fillna_values_v2
)

In [ ]:
#missing_values = integrated_training_df[feature_columns_2].isna().sum() / len(integrated_training_df) * 100
#missing_values

In [ ]:
integrated_training_df_clean[:3]

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,mean_age,either_in_complex,mean_complex_essentiality,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,226.1,False,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702


In [97]:
#summary of the training dataset after removing NA values
print('Num SL:', integrated_training_df_clean[integrated_training_df_clean[target_column] == True].shape[0], '/', integrated_training_df_clean.shape[0])
print('Num non-SL:', integrated_training_df_clean[integrated_training_df_clean[target_column] == False].shape[0], '/', integrated_training_df_clean.shape[0])
print(f'Number of unique gene pairs: {integrated_training_df_clean.genepair.nunique()}')
print(f'Number of unique cell lines: {integrated_training_df_clean.cell_line.nunique()}')
training_df[:3]

Num SL: 958 / 41244
Num non-SL: 40286 / 41244
Number of unique gene pairs: 4170
Number of unique cell lines: 10


,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,A2_rank,zA2_rank,max_ranked_A1A2,min_ranked_A1A2,z_max_ranked_A1A2,z_min_ranked_A1A2,prediction_score,GEMINI,LFC,SL_new
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,NaN,NaN,5108.0,5108.0,-0.759628,-0.759628,0.012559,0.118768,0.088856,False
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,NaN,NaN,9125.0,9125.0,0.303431,0.303431,0.012559,0.132501,0.201704,False
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,NaN,NaN,4063.0,4063.0,-1.036177,-1.036177,0.012559,0.024593,0.069772,False


In [99]:
integrated_training_df_clean.to_csv(get_data_path(['SL PRED', 'NEW'], 'data_SL_pred.csv'), index=False)

In [ ]:
#impute_features = ['ranked_Essentiality_weighted_PPI', 'Expression_weighted_PPI', 'smallest_GO_ranked_ess', 'smallest_GO_CC_ranked_ess', 'smallest_gene_expression', 'smallest_GO_CC_expression']

#integrated_training_df_clean = integrated_training_df.copy()

#integrated_training_df_clean[impute_features] = integrated_training_df_clean.groupby("DepMap_ID")[impute_features].transform(lambda x: x.fillna(x.mean()))

In [16]:
#integrated_training_df_clean.loc[integrated_training_df_clean['max_ranked_A1A2'].isna(), ['genepair','cell_line','max_ranked_A1A2', 'min_ranked_A1A2', 'ranked_Essentiality_weighted_PPI', 'smallest_GO_ranked_ess']] 

In [ ]:
#remove NA values before training the model
#integrated_training_df_clean = integrated_training_df_clean.dropna(axis=0, how='any', subset=feature_columns_2 + [target_column]).reset_index(drop=True) 

#integrated_training_df_clean = integrated_training_df.copy()

#summary of the training dataset after removing NA values
#print('Num SL:', integrated_training_df_clean[integrated_training_df_clean[target_column] == True].shape[0], '/', integrated_training_df_clean.shape[0])
#print('Num non-SL:', integrated_training_df_clean[integrated_training_df_clean[target_column] == False].shape[0], '/', integrated_training_df_clean.shape[0])
#print(f'Number of unique gene pairs: {integrated_training_df_clean.genepair.nunique()}')
#print(f'Number of unique cell lines: {integrated_training_df_clean.cell_line.nunique()}')
#integrated_training_df_clean[:3]

Num SL: 946 / 41114
Num non-SL: 40168 / 41114
Number of unique gene pairs: 4157
Number of unique cell lines: 10


,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,mean_complex_essentiality,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,min_sequence_identity,shared_ppi_mean_essentiality
0,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.340483,0.0
1,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.340483,0.0
2,A3GALT2_ABO,A3GALT2,ABO,127550,28,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,0.0,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.340483,0.0


In [41]:
integrated_training_df_clean[feature_columns_1].isna().sum()

rMaxExp_A1A2                        0
rMinExp_A1A2                        0
max_ranked_A1A2                     0
min_ranked_A1A2                     0
max_cn                              0
min_cn                              0
Protein_Altering                    0
Damaging                            0
min_sequence_identity               0
prediction_score                    0
ranked_Essentiality_weighted_PPI    0
Expression_weighted_PPI             0
smallest_GO_ranked_ess              0
smallest_GO_CC_ranked_ess           0
smallest_gene_expression            0
smallest_GO_CC_expression           0
dtype: int64

In [27]:
#missing_values = integrated_training_df_clean[feature_columns_1].isna().sum() / len(integrated_training_df_clean) * 100
#missing_values

In [ ]:
#integrated_training_df_clean.to_csv(get_data_path(['SL PRED', 'output', 'NEW'], 'integrated_training_df.csv'), index=False)

### Model I - Random Selection

In [42]:
def safe_roc_auc(y_true, y_score):
    mask = ~np.isnan(y_score)
    if np.sum(mask) < 2 or len(np.unique(y_true[mask])) < 2:
        return np.nan
    return roc_auc_score(y_true[mask], y_score[mask])

def safe_average_precision(y_true, y_score):
    mask = ~np.isnan(y_score)
    if np.sum(mask) == 0:
        return np.nan
    return average_precision_score(y_true[mask], y_score[mask])

In [50]:
def model_early_cross_validation(classifier, data, target, splits, verbose=True):
    tprs = []
    fprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    pred_aucs, seq_aucs, gene_expr_aucs, gene_ess_aucs = [], [], [], []
    aps, pred_aps, seq_aps, gene_expr_aps, gene_ess_aps = [], [], [], [], []

    y_real = []
    y_proba = []

    total_splits = len(splits)
    start_time = time.time()

    for fold_num, (train, test) in enumerate(splits, start=1):
        print(f"Fold {fold_num}: Train size = {len(train)}, Test size = {len(test)}")

        if len(test) == 0:
            continue

        y_test = target.iloc[test].values
        y_pred_proba = classifier.fit(data.iloc[train], target.iloc[train]).predict_proba(data.iloc[test])[:, 1]

        if np.unique(y_test).size < 2:
            print(f"Skipping fold {fold_num} due to insufficient positive/negative samples.")
            continue

        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        tprs.append(tpr)
        fprs.append(fpr)

        aucs.append(roc_auc_score(y_test, y_pred_proba))
        aps.append(average_precision_score(y_test, y_pred_proba))

        # Scores from input data columns
        pred_aucs.append(safe_roc_auc(y_test, data.iloc[test]['prediction_score'].values))
        seq_aucs.append(safe_roc_auc(y_test, data.iloc[test]['min_sequence_identity'].values))
        gene_expr_aucs.append(safe_roc_auc(y_test, data.iloc[test]['rMinExp_A1A2'].values))
        gene_ess_aucs.append(1 - safe_roc_auc(y_test, data.iloc[test]['min_ranked_A1A2'].values))

        pred_aps.append(safe_average_precision(y_test, data.iloc[test]['prediction_score'].values))
        seq_aps.append(safe_average_precision(y_test, data.iloc[test]['min_sequence_identity'].values))
        gene_expr_aps.append(safe_average_precision(y_test, data.iloc[test]['rMinExp_A1A2'].values))
        gene_ess_aps.append(safe_average_precision(y_test, data.iloc[test]['min_ranked_A1A2'].values))

        y_real.append(y_test)
        y_proba.append(y_pred_proba)

        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Fold {fold_num}/{total_splits} complete: ROC AUC = {aucs[-1]:.4f}, "
                  f"Elapsed time = {elapsed_time:.2f} seconds")

    if len(tprs) > 0:
        mean_tpr = np.mean([np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(tprs))], axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
    else:
        mean_tpr, mean_auc, std_auc = np.array([]), np.nan, np.nan

    if len(y_real) > 0 and len(y_proba) > 0:
        y_real = np.concatenate(y_real)
        y_proba = np.concatenate(y_proba)
        precision, recall, _ = precision_recall_curve(y_real, y_proba)
    else:
        precision, recall = np.array([]), np.array([])

    if verbose and not np.isnan(mean_auc):
        print(f"Cross-validation complete: Mean ROC AUC = {mean_auc:.4f}, Std ROC AUC = {std_auc:.4f}")

    return {
        'tprs': tprs, 'fprs': fprs, 'aucs': aucs,
        'mean_tpr': mean_tpr, 'mean_fpr': mean_fpr, 'mean_auc': mean_auc, 'std_auc': std_auc,
        'seq_auc': np.nanmean(seq_aucs), 'seq_std_auc': np.nanstd(seq_aucs),
        'pred_auc': np.nanmean(pred_aucs), 'pred_std_auc': np.nanstd(pred_aucs),
        'gene_expr_auc': np.nanmean(gene_expr_aucs), 'gene_expr_std_auc': np.nanstd(gene_expr_aucs),
        'gene_ess_auc': np.nanmean(gene_ess_aucs), 'gene_ess_std_auc': np.nanstd(gene_ess_aucs),
        'precision': precision, 'recall': recall, 'aps': aps,
        'mean_aps': np.nanmean(aps), 'std_ap': np.nanstd(aps),
        'pred_ap': np.nanmean(pred_aps), 'pred_std_ap': np.nanstd(pred_aps),
        'seq_ap': np.nanmean(seq_aps), 'seq_std_ap': np.nanstd(seq_aps),
        'gene_expr_ap': np.nanmean(gene_expr_aps), 'gene_expr_std_ap': np.nanstd(gene_expr_aps),
        'gene_ess_ap': np.nanmean(gene_ess_aps), 'gene_ess_std_ap': np.nanstd(gene_ess_aps)
    }


In [51]:
def model_late_cross_validation(classifier, data, target, splits, verbose=True):
    tprs = []
    fprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    y_real = []
    y_proba = []
    aps = []
    
    total_splits = len(splits)
    start_time = time.time()

    for fold_num, (train, test) in enumerate(splits, start=1):
        print(f"Fold {fold_num}: Train size = {len(train)}, Test size = {len(test)}")
        
        if len(test) == 0:
            continue

        y_test = target.iloc[test].values
        y_pred_proba = classifier.fit(data.iloc[train], target.iloc[train]).predict_proba(data.iloc[test])[:, 1]

        if np.unique(y_test).size < 2:
            print(f"Skipping fold {fold_num} due to insufficient positive/negative samples.")
            continue

        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        tprs.append(tpr)
        fprs.append(fpr)

        aucs.append(safe_roc_auc(y_test, y_pred_proba))
        aps.append(safe_average_precision(y_test, y_pred_proba))

        y_real.append(y_test)
        y_proba.append(y_pred_proba)

        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Fold {fold_num}/{total_splits} complete: ROC AUC = {aucs[-1]:.4f}, "
                  f"Elapsed time = {elapsed_time:.2f} seconds")
    
    if len(tprs) > 0:
        mean_tpr = np.mean([np.interp(mean_fpr, fprs[i], tprs[i]) for i in range(len(tprs))], axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
    else:
        mean_tpr, mean_auc, std_auc = np.array([]), np.nan, np.nan
    
    if len(y_real) > 0 and len(y_proba) > 0:
        y_real = np.concatenate(y_real)
        y_proba = np.concatenate(y_proba)
        precision, recall, _ = precision_recall_curve(y_real, y_proba)
    else:
        precision, recall = np.array([]), np.array([])

    if verbose and not np.isnan(mean_auc):
        print(f"Cross-validation complete: Mean ROC AUC = {mean_auc:.4f}, Std ROC AUC = {std_auc:.4f}")

    return {
        'tprs': tprs, 'fprs': fprs, 'aucs': aucs,
        'mean_tpr': mean_tpr, 'mean_fpr': mean_fpr, 'mean_auc': mean_auc, 'std_auc': std_auc,
        'precision': precision, 'recall': recall, 'aps': aps,
        'mean_aps': np.nanmean(aps), 
        'std_ap': np.nanstd(aps)
    }


In [52]:
# Define feature sets
data_1 = integrated_training_df_clean[feature_columns_1]
data_2 = integrated_training_df_clean[feature_columns_2]
target = integrated_training_df_clean[target_column]

# Define your Random Forest classifier
RF = RandomForestClassifier(n_estimators=600, random_state=8, max_features=0.2, max_depth=20, min_samples_leaf=4)

In [53]:
# Define number of folds
nfolds = 5

# Generate StratifiedKFold splits
kf = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=42)
splits_I = list(kf.split(integrated_training_df_clean[feature_columns_1], integrated_training_df_clean[target_column]))

In [54]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/NEW/imputation/cross_validation/model_I/splits.pkl', 'wb') as f:
    pickle.dump(splits_I, f)

In [55]:
#Run cross-validation on both feature set
model_I_early = model_early_cross_validation(RF, data_1, target, splits_I)
model_I_late = model_late_cross_validation(RF, data_2, target, splits_I)

Fold 1: Train size = 32995, Test size = 8249
Fold 1/5 complete: ROC AUC = 0.9101, Elapsed time = 48.08 seconds
Fold 2: Train size = 32995, Test size = 8249
Fold 2/5 complete: ROC AUC = 0.9180, Elapsed time = 92.66 seconds
Fold 3: Train size = 32995, Test size = 8249
Fold 3/5 complete: ROC AUC = 0.9246, Elapsed time = 137.19 seconds
Fold 4: Train size = 32995, Test size = 8249
Fold 4/5 complete: ROC AUC = 0.9158, Elapsed time = 182.06 seconds
Fold 5: Train size = 32996, Test size = 8248
Fold 5/5 complete: ROC AUC = 0.9283, Elapsed time = 226.58 seconds
Cross-validation complete: Mean ROC AUC = 0.9186, Std ROC AUC = 0.0064
Fold 1: Train size = 32995, Test size = 8249
Fold 1/5 complete: ROC AUC = 0.9203, Elapsed time = 63.83 seconds
Fold 2: Train size = 32995, Test size = 8249
Fold 2/5 complete: ROC AUC = 0.9346, Elapsed time = 132.41 seconds
Fold 3: Train size = 32995, Test size = 8249
Fold 3/5 complete: ROC AUC = 0.9395, Elapsed time = 199.64 seconds
Fold 4: Train size = 32995, Test siz

In [56]:
# save results of cross validation
with open(file_RF_model_I_early, 'wb') as f:
   pickle.dump(model_I_early, f)


with open(file_RF_model_I_late, 'wb') as f:
    pickle.dump(model_I_late, f)

In [ ]:
# Define RepeatedStratifiedKFold with 5 folds and 20 repeats
#nfolds = 5
#n_repeats = 10

#rskf = RepeatedStratifiedKFold(n_splits=nfolds, n_repeats=n_repeats, random_state=42)
#repeated_splits_I = list(rskf.split(integrated_training_df_clean[feature_columns_1], integrated_training_df_clean[target_column]))

In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_I/repeated_splits_I.pkl', 'wb') as f:
    pickle.dump(repeated_splits_I, f)

In [ ]:
#Run cross-validation on both feature set
model_I_early_repeated = model_early_cross_validation(RF, data_1, target, repeated_splits_I)
model_I_late_repeated = model_late_cross_validation(RF, data_2, target, repeated_splits_I)

In [ ]:
# save results of cross validation
with open(file_RF_model_I_early_repeated, 'wb') as f:
    pickle.dump(model_I_early_repeated, f)


with open(file_RF_model_I_late_repeated, 'wb') as f:
    pickle.dump(model_I_late_repeated, f)

In [ ]:
# Define the number of rows to keep in the reduced training set
desired_train_size = 26130

def reduce_train_size(train_index, desired_size):

    # If the current training set size is larger than desired, randomly sample
    if len(train_index) > desired_size:
        reduced_train_index = random.sample(list(train_index), desired_size)
        return reduced_train_index
    else:
        return train_index

# Repeated Stratified K-Fold with train size reduction
def repeated_stratified_reduced_cv(data, target, n_folds, n_repeats, desired_train_size, random_state=42):
    all_reduced_splits = []
    rng = random.Random(random_state)  # Ensure reproducibility with the same random state

    for repeat in range(n_repeats):
        # Generate StratifiedKFold splits for each repeat
        kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state + repeat)
        splits = list(kf.split(data, target))

        # Reduce the training size for each fold and store the result
        reduced_splits = []
        for fold_num, (train_index, test_index) in enumerate(splits, start=1):
            # Reduce the train size to the desired number of rows (26,000 in this case)
            reduced_train_index = reduce_train_size(train_index, desired_train_size)
            
            # Store the reduced train and original test indices
            reduced_splits.append((reduced_train_index, test_index))

            # Print progress
            print(f"Repeat {repeat + 1}, Fold {fold_num} - Train size reduced to: {len(reduced_train_index)}, Test size: {len(test_index)}")

        # Add the reduced splits for this repeat to the overall list
        all_reduced_splits.extend(reduced_splits)

    return all_reduced_splits

reduced_splits_I = repeated_stratified_reduced_cv(
    data=integrated_training_df_clean[feature_columns_1],
    target=integrated_training_df_clean[target_column],
    n_folds=5,
    n_repeats=10,
    desired_train_size=desired_train_size,
    random_state=42
)


In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_I/reduced_splits_I.pkl', 'wb') as f:
    pickle.dump(reduced_splits_I, f)

In [ ]:
#Run cross-validation on both feature set
model_I_early_reduced = model_early_cross_validation(RF, data_1, target, reduced_splits_I)
model_I_late_reduced = model_late_cross_validation(RF, data_2, target, reduced_splits_I)

In [ ]:
# save results of cross validation
with open(file_RF_model_I_early_reduced, 'wb') as f:
    pickle.dump(model_I_early_reduced, f)


with open(file_RF_model_I_late_reduced, 'wb') as f:
    pickle.dump(model_I_late_reduced, f)

## Model II
#### Same paralog pairs, different cell lines

In [67]:
# Define group feature for cell lines
groups = integrated_training_df_clean['cell_line']

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
splits_II = list(sgkf.split(integrated_training_df_clean, target, groups))

for i, (train_index, test_index) in enumerate(splits_II):
    print(f'Fold {i+1}:')
    print(f"TRAIN: group={groups[train_index].unique()}")
    print(f"TEST: group={groups[test_index].unique()}")


Fold 1:
TRAIN: group=['PK1_PANCREAS' 'A549_LUNG' 'GI1_CENTRAL_NERVOUS_SYSTEM' 'HS936T_SKIN'
 'MELJUSO_SKIN' 'IPC298_SKIN' 'HSC5_SKIN' 'MEL202_UVEA']
TEST: group=['PATU8988S_PANCREAS' 'HS944T_SKIN']
Fold 2:
TRAIN: group=['PATU8988S_PANCREAS' 'PK1_PANCREAS' 'HS944T_SKIN' 'A549_LUNG'
 'GI1_CENTRAL_NERVOUS_SYSTEM' 'HS936T_SKIN' 'MELJUSO_SKIN' 'MEL202_UVEA']
TEST: group=['IPC298_SKIN' 'HSC5_SKIN']
Fold 3:
TRAIN: group=['PATU8988S_PANCREAS' 'PK1_PANCREAS' 'HS944T_SKIN'
 'GI1_CENTRAL_NERVOUS_SYSTEM' 'HS936T_SKIN' 'MELJUSO_SKIN' 'IPC298_SKIN'
 'HSC5_SKIN']
TEST: group=['A549_LUNG' 'MEL202_UVEA']
Fold 4:
TRAIN: group=['PATU8988S_PANCREAS' 'HS944T_SKIN' 'A549_LUNG'
 'GI1_CENTRAL_NERVOUS_SYSTEM' 'MELJUSO_SKIN' 'IPC298_SKIN' 'HSC5_SKIN'
 'MEL202_UVEA']
TEST: group=['PK1_PANCREAS' 'HS936T_SKIN']
Fold 5:
TRAIN: group=['PATU8988S_PANCREAS' 'PK1_PANCREAS' 'HS944T_SKIN' 'A549_LUNG'
 'HS936T_SKIN' 'IPC298_SKIN' 'HSC5_SKIN' 'MEL202_UVEA']
TEST: group=['GI1_CENTRAL_NERVOUS_SYSTEM' 'MELJUSO_SKIN']


In [68]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/NEW/imputation/cross_validation/model_II/splits_II.pkl', 'wb') as f:
    pickle.dump(splits_II, f)

In [69]:
model_II_early = model_early_cross_validation(RF, data_1, target, splits_II)
model_II_late = model_late_cross_validation(RF, data_2, target, splits_II)

Fold 1: Train size = 32973, Test size = 8271
Fold 1/5 complete: ROC AUC = 0.9584, Elapsed time = 43.68 seconds
Fold 2: Train size = 32978, Test size = 8266
Fold 2/5 complete: ROC AUC = 0.9263, Elapsed time = 86.87 seconds
Fold 3: Train size = 33018, Test size = 8226
Fold 3/5 complete: ROC AUC = 0.8812, Elapsed time = 131.64 seconds
Fold 4: Train size = 32992, Test size = 8252
Fold 4/5 complete: ROC AUC = 0.8847, Elapsed time = 174.23 seconds
Fold 5: Train size = 33015, Test size = 8229
Fold 5/5 complete: ROC AUC = 0.8572, Elapsed time = 216.51 seconds
Cross-validation complete: Mean ROC AUC = 0.9012, Std ROC AUC = 0.0361
Fold 1: Train size = 32973, Test size = 8271
Fold 1/5 complete: ROC AUC = 0.9694, Elapsed time = 63.73 seconds
Fold 2: Train size = 32978, Test size = 8266
Fold 2/5 complete: ROC AUC = 0.9426, Elapsed time = 126.94 seconds
Fold 3: Train size = 33018, Test size = 8226
Fold 3/5 complete: ROC AUC = 0.9054, Elapsed time = 193.27 seconds
Fold 4: Train size = 32992, Test siz

In [70]:
# save results of cross validation
with open(file_RF_model_II_early, 'wb') as f:
    pickle.dump(model_II_early, f)


with open(file_RF_model_II_late, 'wb') as f:
    pickle.dump(model_II_late, f)

In [ ]:
def repeated_stratified_group_kfold(data, target, groups, n_folds, n_repeats=n_repeats, random_state=42):
    
    all_splits = []
    
    for repeat in range(n_repeats):
        # Set up StratifiedGroupKFold with an incremented random state for each repeat
        sgkf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=random_state + repeat)
        splits = list(sgkf.split(data, target, groups))
        
        # Append each split (train_index, test_index) to all_splits
        for fold_num, (train_index, test_index) in enumerate(splits, start=1):
            all_splits.append((train_index, test_index))
            
            # Print fold details for tracking
            print(f"Repeat {repeat + 1}, Fold {fold_num}:")
            print(f"  TRAIN groups: {groups.iloc[train_index].unique()}")
            print(f"  TEST groups: {groups.iloc[test_index].unique()}")

    return all_splits

# Generate the repeated stratified group k-fold splits
repeated_splits_II = repeated_stratified_group_kfold(
    data=integrated_training_df_clean,
    target=target,
    groups=groups,
    n_folds=5,
    n_repeats=10,
    random_state=42
)

In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_II/repeated_splits_II.pkl', 'wb') as f:
    pickle.dump(repeated_splits_II, f)

In [ ]:
#Run cross-validation on both feature set
model_II_early_repeated = model_early_cross_validation(RF, data_1, target, repeated_splits_II)
model_II_late_repeated = model_late_cross_validation(RF, data_2, target, repeated_splits_II)

In [ ]:
# save results of cross validation
with open(file_RF_model_II_early_repeated, 'wb') as f:
    pickle.dump(model_II_early_repeated, f)


with open(file_RF_model_II_late_repeated, 'wb') as f:
    pickle.dump(model_II_late_repeated, f)

In [ ]:
# Define the number of rows to keep in the reduced training set
desired_train_size = 26130

def reduce_train_size(train_index, desired_size):

    # If the current training set size is larger than desired, randomly sample
    if len(train_index) > desired_size:
        reduced_train_index = random.sample(list(train_index), desired_size)
        return reduced_train_index
    else:
        return train_index

# Repeated Stratified Group K-Fold with train size reduction
def repeated_stratified_group_reduced_cv(data, target, groups, n_folds, n_repeats, desired_train_size, random_state=42):
    all_reduced_splits = []
    rng = random.Random(random_state)  # Ensure reproducibility with the same random state

    for repeat in range(n_repeats):
        sgkf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=random_state + repeat)
        splits = list(sgkf.split(data, target, groups))

        # Reduce the training size for each fold and store the result
        reduced_splits = []
        for fold_num, (train_index, test_index) in enumerate(splits, start=1):
            # Reduce the train size to the desired number of rows (26,000 in this case)
            reduced_train_index = reduce_train_size(train_index, desired_train_size)
            
            # Store the reduced train and original test indices
            reduced_splits.append((reduced_train_index, test_index))

            # Print progress
            print(f"Repeat {repeat + 1}, Fold {fold_num} - Train size reduced to: {len(reduced_train_index)}, Test size: {len(test_index)}")
            print(f"  TRAIN groups: {groups.iloc[reduced_train_index].unique()}")
            print(f"  TEST groups: {groups.iloc[test_index].unique()}")

        # Add the reduced splits for this repeat to the overall list
        all_reduced_splits.extend(reduced_splits)

    return all_reduced_splits

reduced_splits_II = repeated_stratified_group_reduced_cv(
    data=integrated_training_df_clean,
    target=target,
    groups=groups,
    n_folds=5,
    n_repeats=10,
    desired_train_size=desired_train_size,
    random_state=42
)

In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_II/reduced_splits_II.pkl', 'wb') as f:
    pickle.dump(reduced_splits_II, f)

In [ ]:
model_II_early_reduced = model_early_cross_validation(RF, data_1, target, reduced_splits_II)
model_II_late_reduced = model_late_cross_validation(RF, data_2, target, reduced_splits_II)

In [ ]:
# save results of cross validation
with open(file_RF_model_II_early_reduced, 'wb') as f:
    pickle.dump(model_II_early_reduced, f)


with open(file_RF_model_II_late_reduced, 'wb') as f:
    pickle.dump(model_II_late_reduced, f)

## Model III
#### Different paralog pairs, same cell lines

In [75]:
# Define group feature for gene pairs
gene_groups = integrated_training_df_clean['genepair']

sgkf2 = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
splits_III = list(sgkf.split(integrated_training_df_clean, target, gene_groups))

for i, (train_index, test_index) in enumerate(splits_III):
    print(f'Fold {i+1}:')
    print(f"TRAIN: group={groups[train_index].nunique()}")
    print(f"TEST: group={groups[test_index].nunique()}")
    print(f'# of overlapping pairs: {np.isin(groups[train_index].unique(), groups[test_index].unique()).sum()}') # True is 1, False is 0


Fold 1:
TRAIN: group=10
TEST: group=10
# of overlapping pairs: 10
Fold 2:
TRAIN: group=10
TEST: group=10
# of overlapping pairs: 10
Fold 3:
TRAIN: group=10
TEST: group=10
# of overlapping pairs: 10
Fold 4:
TRAIN: group=10
TEST: group=10
# of overlapping pairs: 10
Fold 5:
TRAIN: group=10
TEST: group=10
# of overlapping pairs: 10


In [76]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/NEW/imputation/cross_validation/model_III/splits_III.pkl', 'wb') as f:
    pickle.dump(splits_III, f)

In [77]:
model_III_early = model_early_cross_validation(RF, data_1, target, splits_III)
model_III_late = model_late_cross_validation(RF, data_2, target, splits_III)

Fold 1: Train size = 33027, Test size = 8217
Fold 1/5 complete: ROC AUC = 0.8739, Elapsed time = 48.31 seconds
Fold 2: Train size = 32987, Test size = 8257
Fold 2/5 complete: ROC AUC = 0.9121, Elapsed time = 106.06 seconds
Fold 3: Train size = 32975, Test size = 8269
Fold 3/5 complete: ROC AUC = 0.9168, Elapsed time = 158.55 seconds
Fold 4: Train size = 32987, Test size = 8257
Fold 4/5 complete: ROC AUC = 0.9171, Elapsed time = 211.62 seconds
Fold 5: Train size = 33000, Test size = 8244
Fold 5/5 complete: ROC AUC = 0.9050, Elapsed time = 265.65 seconds
Cross-validation complete: Mean ROC AUC = 0.9043, Std ROC AUC = 0.0161
Fold 1: Train size = 33027, Test size = 8217
Fold 1/5 complete: ROC AUC = 0.8865, Elapsed time = 77.53 seconds
Fold 2: Train size = 32987, Test size = 8257
Fold 2/5 complete: ROC AUC = 0.9078, Elapsed time = 153.94 seconds
Fold 3: Train size = 32975, Test size = 8269
Fold 3/5 complete: ROC AUC = 0.9218, Elapsed time = 233.43 seconds
Fold 4: Train size = 32987, Test si

In [78]:
# save results of cross validation
with open(file_RF_model_III_early, 'wb') as f:
    pickle.dump(model_III_early, f)


with open(file_RF_model_III_late, 'wb') as f:
    pickle.dump(model_III_late, f)

In [ ]:
# Generate the repeated stratified group k-fold splits

repeated_splits_III = repeated_stratified_group_kfold(
    data=integrated_training_df_clean,
    target=target,
    groups=gene_groups,
    n_folds=5,
    n_repeats=10,
    random_state=42
)

In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_III/repeated_splits_III.pkl', 'wb') as f:
    pickle.dump(repeated_splits_III, f)

In [ ]:
#Run cross-validation on both feature set
model_III_early_repeated = model_early_cross_validation(RF, data_1, target, repeated_splits_III)
model_III_late_repeated = model_late_cross_validation(RF, data_2, target, repeated_splits_III)

In [ ]:
# save results of cross validation
with open(file_RF_model_III_early_repeated, 'wb') as f:
    pickle.dump(model_III_early_repeated, f)


with open(file_RF_model_III_late_repeated, 'wb') as f:
    pickle.dump(model_III_late_repeated, f)

In [ ]:
# Define the number of rows to keep in the reduced training set
desired_train_size = 26130

reduced_splits_III = repeated_stratified_group_reduced_cv(
    data=integrated_training_df_clean,
    target=target,
    groups=gene_groups,
    n_folds=5,
    n_repeats=10,
    desired_train_size=desired_train_size,
    random_state=42
)

In [ ]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/output/cross_validation/model_III/reduced_splits_III.pkl', 'wb') as f:
    pickle.dump(reduced_splits_III, f)

In [ ]:
model_III_early_reduced = model_early_cross_validation(RF, data_1, target, reduced_splits_III)
model_III_late_reduced = model_late_cross_validation(RF, data_2, target, reduced_splits_III)

In [ ]:
# save results of cross validation
with open(file_RF_model_III_early_reduced, 'wb') as f:
    pickle.dump(model_III_early_reduced, f)


with open(file_RF_model_III_late_reduced, 'wb') as f:
    pickle.dump(model_III_late_reduced, f)

## Model IV
#### Different paralog pairs, different cell lines

In [62]:
# --- Safe metric wrappers ---
def safe_roc_auc(y_true, y_score):
    mask = ~np.isnan(y_score)
    if np.sum(mask) < 2 or len(np.unique(y_true[mask])) < 2:
        return np.nan
    return roc_auc_score(y_true[mask], y_score[mask])

def safe_average_precision(y_true, y_score):
    mask = ~np.isnan(y_score)
    if np.sum(mask) == 0:
        return np.nan
    return average_precision_score(y_true[mask], y_score[mask])

# --- Custom split functions ---
def create_disjoint_splits(df, n_splits=5, num_test_cell_lines=2, test_gene_fraction=0.25, random_state=None):
    splits = []

    # Get the unique cell lines and gene pairs
    unique_cell_lines = df['DepMap_ID'].unique()
    unique_gene_pairs = df['genepair'].unique()

    # Set the random seed if provided
    if random_state is not None:
        np.random.seed(random_state)

    # Shuffle and split cell lines and gene pairs into mutually exclusive groups
    np.random.shuffle(unique_cell_lines)
    cell_line_splits = np.array_split(unique_cell_lines, n_splits)

    np.random.shuffle(unique_gene_pairs)
    gene_pair_splits = np.array_split(unique_gene_pairs, n_splits)

    # Create the splits
    for fold in range(n_splits):
        test_cell_lines = cell_line_splits[fold]
        test_gene_pairs = gene_pair_splits[fold]

        train_cell_lines = np.concatenate([cell_line_splits[i] for i in range(n_splits) if i != fold])
        train_gene_pairs = np.concatenate([gene_pair_splits[i] for i in range(n_splits) if i != fold])

        # Define train/test indices
        test_index = df[(df['DepMap_ID'].isin(test_cell_lines)) & (df['genepair'].isin(test_gene_pairs))].index
        train_index = df[(df['DepMap_ID'].isin(train_cell_lines)) & (df['genepair'].isin(train_gene_pairs))].index

        splits.append((train_index, test_index))

        print(f'[Fold {fold+1}] '
              f'# pairs (train): {df.loc[train_index, "genepair"].nunique()} | '
              f'# pairs (test): {df.loc[test_index, "genepair"].nunique()} | '
              f'# overlapping: {np.isin(df.loc[test_index, "genepair"].unique(), df.loc[train_index, "genepair"].unique()).sum()} | '
              f'# cells (train): {df.loc[train_index, "DepMap_ID"].nunique()} | '
              f'# cells (test): {df.loc[test_index, "DepMap_ID"].nunique()}')

    return splits

def repeated_custom_cv(df, n_splits=5, n_repeats=3, num_test_cell_lines=2, test_gene_fraction=0.25, random_state=None):
    all_splits = []

    for repeat in range(n_repeats):
        current_seed = random_state + repeat if random_state is not None else None
        splits = create_disjoint_splits(
            df,
            n_splits=n_splits,
            num_test_cell_lines=num_test_cell_lines,
            test_gene_fraction=test_gene_fraction,
            random_state=current_seed
        )
        all_splits.extend(splits)

    return all_splits


In [63]:
splits_IV = repeated_custom_cv(df=integrated_training_df_clean, n_splits=5, n_repeats=10, random_state=42)

[Fold 1] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 2] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 3] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 4] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 5] # pairs (train): 3336 | # pairs (test): 830 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 1] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 2] # pairs (train): 3336 | # pairs (test): 832 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 3] # pairs (train): 3336 | # pairs (test): 833 | # overlapping: 0 | # cells (train): 8 | # cells (test): 2
[Fold 4] # pairs (train): 3336 | # pairs (test): 834 | # overlapping: 0 | # cells (train): 8 | #

In [64]:
# Save splits
with open('/Users/narod/Library/CloudStorage/GoogleDrive-narod.kebabci@ucdconnect.ie/My Drive/SL PRED/NEW/imputation/cross_validation/model_IV/splits_IV.pkl', 'wb') as f:
    pickle.dump(splits_IV, f)

In [65]:
model_IV_early = model_early_cross_validation(RF, data_1, target, splits_IV)
model_IV_late = model_late_cross_validation(RF, data_2, target, splits_IV)

Fold 1: Train size = 26389, Test size = 1651
Fold 1/50 complete: ROC AUC = 0.8909, Elapsed time = 34.30 seconds
Fold 2: Train size = 26403, Test size = 1651
Fold 2/50 complete: ROC AUC = 0.8416, Elapsed time = 72.95 seconds
Fold 3: Train size = 26371, Test size = 1659
Fold 3/50 complete: ROC AUC = 0.8924, Elapsed time = 113.43 seconds
Fold 4: Train size = 26358, Test size = 1658
Fold 4/50 complete: ROC AUC = 0.8562, Elapsed time = 153.84 seconds
Fold 5: Train size = 26465, Test size = 1635
Fold 5/50 complete: ROC AUC = 0.9318, Elapsed time = 190.60 seconds
Fold 6: Train size = 26421, Test size = 1638
Fold 6/50 complete: ROC AUC = 0.8349, Elapsed time = 231.25 seconds
Fold 7: Train size = 26410, Test size = 1645
Fold 7/50 complete: ROC AUC = 0.8899, Elapsed time = 270.97 seconds
Fold 8: Train size = 26386, Test size = 1655
Fold 8/50 complete: ROC AUC = 0.8565, Elapsed time = 310.16 seconds
Fold 9: Train size = 26375, Test size = 1652
Fold 9/50 complete: ROC AUC = 0.8711, Elapsed time = 

In [79]:
# save results of cross validation
with open(file_RF_model_IV_early, 'wb') as f:
    pickle.dump(model_IV_early, f)


with open(file_RF_model_IV_late, 'wb') as f:
    pickle.dump(model_IV_late, f)

In [ ]:
a = []
for i in range(50):
    ax = len(splits_IV[i][0])
    a.append(ax)
    mean_a = np.average(a)

mean_a

## Draw bar plots to visualize the class balance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, roc_curve, precision_recall_curve

def plot_model_comparison(df, data, target, cv_results_1, cv_results_2, model_1, model_2):
    """
    Visualizes the ROC and PR curves for two models (Context-Specific and Old Classifier).
    
    Args:
    - df: Original dataframe containing 'SL' column (for baseline calculations).
    - data: Dataframe with feature values for additional classifiers.
    - target: Target column (binary classification).
    - cv_results_1: Cross-validation results for the first model.
    - cv_results_2: Cross-validation results for the second model.
    - model_1: Name of the first model.
    - model_2: Name of the second model.
    """

    sns.set_context('paper')
    f, ax = plt.subplots(1, 2, figsize=(8, 4))  # Set figure size

    # Model 1: ROC & PR curves (Context-Specific Classifier)
    RocCurveDisplay(
        fpr=cv_results_1.get('mean_fpr', []), 
        tpr=cv_results_1.get('mean_tpr', []), 
        roc_auc=cv_results_1.get('mean_auc', np.nan),
        estimator_name=model_1
    ).plot(ax=ax[0], alpha=0.8, color='#E64B35B2')

    PrecisionRecallDisplay(
        precision=cv_results_1.get('precision', []), 
        recall=cv_results_1.get('recall', []), 
        average_precision=cv_results_1.get('mean_aps', np.nan),
        estimator_name=model_1
    ).plot(ax=ax[1], alpha=0.8, color='#E64B35B2')

    # Model 2: ROC & PR curves (Old Classifier)
    RocCurveDisplay(
        fpr=cv_results_2.get('mean_fpr', []), 
        tpr=cv_results_2.get('mean_tpr', []), 
        roc_auc=cv_results_2.get('mean_auc', np.nan),
        estimator_name=model_2
    ).plot(ax=ax[0], alpha=0.8, color='#4DBBD5B2')

    PrecisionRecallDisplay(
        precision=cv_results_2.get('precision', []), 
        recall=cv_results_2.get('recall', []), 
        average_precision=cv_results_2.get('mean_aps', np.nan),
        estimator_name=model_2
    ).plot(ax=ax[1], alpha=0.8, color='#4DBBD5B2')

    # Additional Comparisons: Old Classifier & Sequence Identity
    metrics = ['prediction_score', 'max_seq_id']
    colors = ['#4DBBD5B2', '#EFC000FF']
    labels = ['Old Classifier', 'Sequence Identity']
    auc_labels = ['pred_auc', 'seq_auc']
    ap_labels = ['pred_ap', 'seq_ap']

    for metric, color, label, auc_label, ap_label in zip(metrics, colors, labels, auc_labels, ap_labels):
        if metric not in data.columns:
            continue  # Skip if the metric column does not exist

        # Compute ROC and PR curves
        fpr, tpr, _ = roc_curve(target, data[metric])
        precision, recall, _ = precision_recall_curve(target, data[metric])

        # Plot ROC curve
        RocCurveDisplay(
            fpr=fpr, tpr=tpr, 
            roc_auc=cv_results_1.get(auc_label, np.nan), 
            estimator_name=label
        ).plot(ax=ax[0], alpha=0.8, color=color)

        # Plot PR curve
        PrecisionRecallDisplay(
            precision=precision, recall=recall, 
            average_precision=cv_results_1.get(ap_label, np.nan), 
            estimator_name=label
        ).plot(ax=ax[1], alpha=0.8, color=color)

    # Chance line for ROC
    ax[0].plot([0, 1], [0, 1], color="lightgrey", linestyle="--", label='Chance (0.50)')
    ax[0].set_xlim([-0.025, 1.025])
    ax[0].set_ylim([-0.025, 1.025])
    ax[0].spines.top.set(visible=False)
    ax[0].spines.right.set(visible=False)
    ax[0].set_xlabel('False Positive Rate', fontsize=10)
    ax[0].set_ylabel('True Positive Rate', fontsize=10)
    ax[0].legend(loc='best', fontsize='small')

    # Chance line for PR curve (no skill level based on target class distribution)
    no_skill = round(df['SL'].mean(), 2)
    ax[1].plot([0, 1], [no_skill, no_skill], linestyle='--', color='lightgrey', label=f'Chance ({no_skill:.2f})')
    ax[1].set_xlim([-0.025, 1.025])
    ax[1].set_ylim([-0.025, 1.025])
    ax[1].spines.top.set(visible=False)
    ax[1].spines.right.set(visible=False)
    ax[1].set_xlabel('Recall', fontsize=10)
    ax[1].set_ylabel('Precision', fontsize=10)
    ax[1].legend(loc='best', fontsize='small')

    plt.tight_layout()
    # plt.close(f)  # Uncomment if generating multiple plots to save memory

# Example usage:
# plot_model_comparison(df, data, target, cv_results, cv_results_v2, '/Users/narod/Desktop', 'Context_Model', 'Old_Model')


In [ ]:
plot_model_comparison(integrated_training_df, integrated_training_df[['prediction_score', 'max_seq_id']], integrated_training_df['SL'], cv_results, cv_results_II, 'Early Integration', 'Late Integration')

In [ ]:
def class_balance(df, splits, k, title):
    percentages_train = []
    percentages_test = []
    train_sizes = []
    test_sizes = []
    # Loop through each split and calculate the percentage of True values
    for i in range(k):
    # Get the subset of the DataFrame based on the split indices
        subset1 = df.iloc[splits[i][0]]
        subset2 = df.iloc[splits[i][1]]
        # Calculate the percentage of True values in the subset
        percentage_true_train = (subset1['SL_08'].sum() / len(subset1['SL_08'])) * 100
        percentage_true_test = (subset2['SL_08'].sum() / len(subset2['SL_08'])) * 100
        # Append the percentage to the list
        percentages_train.append(percentage_true_train)
        percentages_test.append(percentage_true_test)
        train_sizes.append(len(subset1))
        test_sizes.append(len(subset2))
    width = 0.35
    x = np.arange(k)
    #define colors
    train_color = '#0072B2'
    test_color = '#E69F00'
    # Create the figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    # Plot 1: Grouped bar chart for percentages
    ax1.bar(x - width/2, percentages_train, width, label='Train', color=train_color)
    ax1.bar(x + width/2, percentages_test, width, label='Test', color=test_color)
    ax1.set_xlabel('Split Number')
    ax1.set_ylabel('Percentage of True Values')
    ax1.set_title('Percentage of True Values in Train and Test Splits:'+ title)
    ax1.set_xticks(x)
    ax1.set_xticklabels([f'Split {i}' for i in range(k)])
    ax1.bar_label(ax1.containers[0], label_type='edge', fmt='%.2f')
    ax1.bar_label(ax1.containers[1], label_type='edge', fmt='%.2f')
    ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=2)
    # Plot 2: Grouped bar chart for number of rows
    ax2.bar(x - width/2, train_sizes, width, label='Train Size', color=train_color)
    ax2.bar(x + width/2, test_sizes, width, label='Test Size', color=test_color)
    ax2.set_xlabel('Split Number')
    ax2.set_ylabel('Number of Rows')
    ax2.set_title('Number of Rows in Train and Test Subsets:' + title)
    ax2.set_xticks(x)
    ax2.set_xticklabels([f'Split {i}' for i in range(k)])
    ax2.bar_label(ax2.containers[0], label_type='edge')
    ax2.bar_label(ax2.containers[1], label_type='edge')
    #ax2.legend()
    ax2.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=2)
    # Adjust layout
    plt.tight_layout()
    # Show the plots
    plt.show()

In [ ]:
class_balance(integrated_training_df_clean, splits_I, 5, "Model I")
class_balance(integrated_training_df, reduced_splits_I, 10, "Model I Reduced")

In [ ]:
class_balance(integrated_training_df_clean, splits_II, 5, "Model 2")
class_balance(integrated_training_df_clean, reduced_splits_II, 10, "Model 2 Reduced")

In [ ]:
class_balance(integrated_training_df_clean, splits_III, 5, "Model 3")
class_balance(integrated_training_df_clean, reduced_splits_III, 10, "Model 3 Reduced")

In [ ]:
class_balance(integrated_training_df_clean, splits_IV, 10, "Model 4")